In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time

In [13]:
# load the balanced data
data_balanced = pd.read_csv('UNSW_NB15_balanced_smote.csv')

In [13]:
df = data_balanced.copy()
df = df.drop(['attack_cat'], axis=1)

### LSTM Model for Binary Classification

In [14]:
# train an lstm model on the data
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
X = df.drop('label', axis=1)
Y = df['label']

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)


# define the model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
start = time.time()
# train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end - start)

Epoch 1/10
23250/23250 [==============================] - 42s 2ms/step - loss: 0.0953 - accuracy: 0.9691 - val_loss: 0.0842 - val_accuracy: 0.9702
Epoch 2/10
23250/23250 [==============================] - 40s 2ms/step - loss: 0.0818 - accuracy: 0.9707 - val_loss: 0.0795 - val_accuracy: 0.9704
Epoch 3/10
23250/23250 [==============================] - 43s 2ms/step - loss: 0.0783 - accuracy: 0.9710 - val_loss: 0.0771 - val_accuracy: 0.9705
Epoch 4/10
23250/23250 [==============================] - 41s 2ms/step - loss: 0.0764 - accuracy: 0.9711 - val_loss: 0.0762 - val_accuracy: 0.9706
Epoch 5/10
23250/23250 [==============================] - 40s 2ms/step - loss: 0.0752 - accuracy: 0.9711 - val_loss: 0.0751 - val_accuracy: 0.9706
Epoch 6/10
23250/23250 [==============================] - 41s 2ms/step - loss: 0.0742 - accuracy: 0.9712 - val_loss: 0.0730 - val_accuracy: 0.9709
Epoch 7/10
23250/23250 [==============================] - 45s 2ms/step - loss: 0.0734 - accuracy: 0.9713 - val_loss: 0

In [16]:
start_time = time.time()    
y_pred = model.predict(X_test)
end_time = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred, digits = 4))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))
print('Time taken to predict: ', end_time - start_time)

5813/5813 [==============================] - 4s 613us/step
Classification Report
              precision    recall  f1-score   support

           0     0.9920    0.7165    0.8320     18731
           1     0.9692    0.9994    0.9840    167269

    accuracy                         0.9709    186000
   macro avg     0.9806    0.8579    0.9080    186000
weighted avg     0.9715    0.9709    0.9687    186000

Accuracy:
0.9708655913978494
Time taken to predict:  4.248834133148193


In [18]:
# save the model
model.save('unsw_binary_lstm_smote.h5')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### Cross Validation for binary LSTM Model

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline


def get_model():
    # define the model
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
    model.add(Dropout(0.1))
    model.add(LSTM(64, return_sequences=True, activation='relu'))
    model.add(Dropout(0.1))
    model.add(LSTM(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=get_model, epochs=10, batch_size=32, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

X = df.drop('label', axis=1)
Y = df[['label']]

X_fit = X.values.reshape(X.shape[0],1 ,X.shape[1])
Y['label'] = Y['label'].astype('int')

results = cross_val_score(estimator, X_fit, Y, cv=kfold)
print("Accuracy: %.5f%%" % (results.mean()*100))